In [1]:
## IMPORT LIBRARIES

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Binomial
from scipy.io import loadmat
import statsmodels.formula.api as smf

from sklearn.model_selection import GridSearchCV, GroupKFold, PredefinedSplit, ParameterGrid
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
import re

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import linprog
from scipy.sparse import kron, eye, vstack, csr_matrix

In [ ]:
## LOAD THE DATA, CLEAN, AND SET THE CROSS-VALIDATION / PREDICTION DATASET

# Load data
data = pd.read_csv('data.csv', index_col=0)
data['category'] = data['category'].replace('online-accommodation', 'online-travel-market')


# Set distance to 0 if the exporter origin is the same as the importer destionation
data.loc[data['iso_d'] == data['iso_o'], 'dist'] = 0


# Find indices where 'trade_value' is not NaN and 'total_export_value' is greater than 10^7
z_vals = data[(~data['trade_value'].isna()) & (data['total_export_value'] > 10**7)].index


# Apply log transformation
data_log = data.copy()
log_transform_columns = ['trade_value', 'total_export_value', 'total_category_value',
                         'iso_o_gdp', 'iso_d_gdp', 'dist',
                         'iso_o_internet', 'iso_d_internet', 'iso_o_fixed_broadband', 
                         'iso_d_fixed_broadband', 'iso_o_mobile_broadband', 'iso_d_mobile_broadband',
                         'revenues_iso_o']

for col in log_transform_columns:
    data_log[col] = np.log(1 + data_log[col])


# Apply categorical transformation
categorical_cols = ['year', 'category', 'iso_o_loc', 'iso_d_loc']
for col in categorical_cols:
    data_log[col] = data_log[col].astype('category')  

# Get the data used for regression
data_regression = data_log.loc[z_vals]

In [ ]:
## REGRESSION TREE CROSS-VALIDATION


# One hot encoding of categorical variables
X = data_regression.copy()

for col in ['iso_o_loc', 'iso_d_loc', 'year']:  # list all your categorical columns
    dummies = pd.get_dummies(data_regression[col], prefix=col, drop_first=False)
    X = pd.concat([X, dummies], axis=1).drop(col, axis=1)

y = data_regression['trade_value']  
X.drop(['category', 'firm', 'iso_o', 'iso_d', 'trade_value'], axis=1, inplace=True)

# Set baseline categories for logistic regression
X_log_reg = X.copy()
X_log_reg.drop(['iso_o_loc_North America', 'iso_d_loc_North America', 'year_2016'], axis=1, inplace=True)

# Create the group variable
groups = data_regression['firm'].astype(str) + '_' + data_regression['iso_o'].astype(str) + '_' + data_regression['category'].astype(str)
groups = groups.astype(str).factorize()[0]

# Initialize GroupKFold
gkf = GroupKFold(n_splits=5)

# Parameter grid for GridSearchCV
param_grid = {'max_depth': [1, 3, 5, 7], 
              'min_child_weight': [5, 20, 50, 100, 200]}

# For storing the results
mse_scores_per_set = {tuple(sorted(param.items())): [] for param in ParameterGrid(param_grid)}
mse_train_scores_per_set = {tuple(sorted(param.items())): [] for param in ParameterGrid(param_grid)}

# Iterate over each fold
fold_num = 0
for train_index, test_index in gkf.split(X, y, groups=groups):
    fold_num += 1
    X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()
    X_log_reg_train, X_log_reg_test = X_log_reg.iloc[train_index].copy(), X_log_reg.iloc[test_index].copy()
    
    y_train, y_test = y.iloc[train_index].copy(), y.iloc[test_index].copy()

    # Fit logistic regression for zeroProbabilities
    log_reg = LogisticRegression(max_iter=2000)
    log_reg.fit(X_log_reg_train, (y_train > 0))
    X_train['zeroProbabilities'] = log_reg.predict_proba(X_log_reg_train)[:, 1]
    X_test['zeroProbabilities'] = log_reg.predict_proba(X_log_reg_test)[:, 1]

    # Manually iterate over each combination of hyperparameters
    for params in ParameterGrid(param_grid):
        
        estimator = XGBRegressor(**params, n_estimators=150, learning_rate=0.10,
                                  random_state=42)
        estimator.fit(X_train, y_train)
        
        y_pred = estimator.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        
        y_fit = estimator.predict(X_train)
        mse_train = mean_squared_error(y_train, y_fit)
        
        # Print fold number, parameters, and MSE
        print(f"Fold {fold_num}, Params: {params}, MSE: {mse}, MSEtrain: {mse_train}")
        
        mse_scores_per_set[tuple(sorted(params.items()))].append(mse)
        mse_train_scores_per_set[tuple(sorted(params.items()))].append(mse_train)


# Calculate average MSE for each parameter set
avg_mse_scores = {params: np.mean(mse_scores) for params, mse_scores in mse_scores_per_set.items()}
avg_mse_train_scores = {params: np.mean(mse_train_scores) for params, mse_train_scores in mse_train_scores_per_set.items()}

# Determine the best parameter set
best_params = min(avg_mse_scores, key=avg_mse_scores.get)
best_score = avg_mse_scores[best_params]
best_train_score = avg_mse_train_scores[best_params]

print("Best parameters based on average MSE across folds:", dict(best_params))
print("Best average MSE:", best_score)
print("Best train average MSE:", best_train_score)

In [ ]:
## PLOT CROSS-VALIDATION HEATMAP

# Assume avg_mse_scores is available in your environment

# Convert to DataFrame and pivot
avg_mse_df = pd.DataFrame.from_dict(avg_mse_scores, orient='index', columns=['avg_mse'])
avg_mse_df['max_depth'] = avg_mse_df.index.map(lambda x: dict(x)['max_depth'])
avg_mse_df['min_child_weight'] = avg_mse_df.index.map(lambda x: dict(x)['min_child_weight'])
avg_mse_pivot = avg_mse_df.pivot('max_depth', 'min_child_weight', 'avg_mse')

# Plotting
plt.figure(figsize=(10, 8))
sns.heatmap(avg_mse_pivot, annot=True, fmt=".3f", cmap="viridis")
plt.title("Average MSE for Each Hyperparameter Combination")
plt.xlabel("Min Child Weight")
plt.ylabel("Max Depth")
plt.show()

In [ ]:
## COMPARE WITH LINEAR REGRESSION

# One hot encoding of categorical variables
X = data_regression.copy()

for col in ['iso_o_loc', 'iso_d_loc', 'year']:  # list all your categorical columns
    dummies = pd.get_dummies(data_regression[col], prefix=col, drop_first=False)
    X = pd.concat([X, dummies], axis=1).drop(col, axis=1)
    
y = data_regression['trade_value']  
X.drop(['category', 'firm', 'iso_o', 'iso_d', 'trade_value'], axis=1, inplace=True)

# Remove baseline categories
X.drop(['iso_o_loc_North America', 'iso_d_loc_North America', 'year_2016'], axis=1, inplace=True)

# For storing MSE values
mse_scores_lin_reg = []

# Iterate over each fold
fold_num = 0
for train_index, test_index in gkf.split(X, y, groups=groups):
    fold_num += 1
    X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()
    y_train, y_test = y.iloc[train_index].copy(), y.iloc[test_index].copy()

    # Fit the linear regression model
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)

    # Predict on the test set and calculate MSE
    y_pred = lin_reg.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    
    # Store and print MSE
    mse_scores_lin_reg.append(mse)

    print(f"Fold {fold_num}, MSE: {mse}")

# Calculate the average MSE across all folds
avg_mse_lin_reg = np.mean(mse_scores_lin_reg)
print(f"Average MSE for Linear Regression: {avg_mse_lin_reg}")


In [ ]:
## GET FINAL PREDICTIONS


X = data_regression.copy()

for col in ['iso_o_loc', 'iso_d_loc', 'year']:  # list all your categorical columns
    dummies = pd.get_dummies(data_regression[col], prefix=col, drop_first=False)
    X = pd.concat([X, dummies], axis=1).drop(col, axis=1)

y = data_regression['trade_value']  
X.drop(['category', 'firm', 'iso_o', 'iso_d', 'trade_value'], axis=1, inplace=True)

X_log_reg = X.copy()
X_log_reg.drop(['iso_o_loc_North America', 'iso_d_loc_North America', 'year_2016'], axis=1, inplace=True)


# Fit logistic regression for zeroProbabilities on all data
log_reg = LogisticRegression(max_iter=2000)
log_reg.fit(X_log_reg, (y > 0))
zero_prob = log_reg.predict_proba(X_log_reg)[:, 1]

# Add zeroProbabilities to the feature set
X['zeroProbabilities'] = zero_prob

X_prediction = data_log.copy()

for col in ['iso_o_loc', 'iso_d_loc', 'year']:  # list all your categorical columns
    dummies = pd.get_dummies(data_log[col], prefix=col, drop_first=False)
    X_prediction = pd.concat([X_prediction, dummies], axis=1).drop(col, axis=1)

X_prediction.drop(['category', 'firm', 'iso_o', 'iso_d', 'trade_value'], axis=1, inplace=True)

X_prediction_log_reg = X_prediction.copy()
X_prediction_log_reg.drop(['iso_o_loc_North America', 'iso_d_loc_North America', 'year_2016'], axis=1, inplace=True)


# Fit logistic regression for zeroProbabilities on all data
zero_prob = log_reg.predict_proba(X_prediction_log_reg)[:, 1]

# Add zeroProbabilities to the feature set
X_prediction['zeroProbabilities'] = zero_prob

# Fit the GBRT
gbr = XGBRegressor(learning_rate=0.1, n_estimators=150,
                       max_depth=best_params[0][1],
                       min_child_weight=best_params[1][1],
                       random_state=42)
gbr.fit(X, y)


# Fit the linear regression model
lin_reg.fit(X_log_reg, y)




data['ypred_combined'] = gbr.predict(X_prediction)
data['ypred_linear_combined'] = lin_reg.predict(X_prediction_log_reg)


columns_to_keep = ['firm', 'year', 'category', 'iso_o', 'iso_d', 'trade_value', 'total_export_value', 'ypred_combined', 'ypred_linear_combined']

# Select only these columns
TT_reduced = data[columns_to_keep].copy()

# Create non-normalized predictions
TT_reduced['trade_value_pred'] = np.exp(TT_reduced['ypred_combined']) - 1
TT_reduced.loc[TT_reduced['trade_value_pred'] < 1000, 'trade_value_pred'] = 0

TT_reduced['trade_value_linear_pred'] = np.exp(TT_reduced['ypred_linear_combined']) - 1
TT_reduced.loc[TT_reduced['trade_value_linear_pred'] < 1000, 'trade_value_linear_pred'] = 0


In [ ]:
## POST PROCESSING # 1 - summing up to firm revenues

# Calculate normalized values for categories other than 'apps' and 'games'
non_app_game_mask = ~TT_reduced['category'].isin(['apps', 'games'])
grouped = TT_reduced[non_app_game_mask].groupby(['firm', 'year', 'category', 'iso_o'])
TT_reduced.loc[non_app_game_mask, 'normalized_trade_value_pred'] = grouped['trade_value_pred'].transform(lambda x: x / x.sum() * TT_reduced.loc[x.index, 'total_export_value'])
TT_reduced.loc[non_app_game_mask, 'normalized_trade_value_linear_pred'] = grouped['trade_value_linear_pred'].transform(lambda x: x / x.sum() * TT_reduced.loc[x.index, 'total_export_value'])

# For 'apps' and 'games' categories, set normalized_trade_value_pred based on the condition
app_game_mask = TT_reduced['category'].isin(['apps', 'games'])
TT_reduced.loc[app_game_mask, 'normalized_trade_value_pred'] = TT_reduced.loc[app_game_mask, 'trade_value'].fillna(TT_reduced.loc[app_game_mask, 'trade_value_pred'])
TT_reduced.loc[app_game_mask, 'normalized_trade_value_linear_pred'] = TT_reduced.loc[app_game_mask, 'trade_value'].fillna(TT_reduced.loc[app_game_mask, 'trade_value_linear_pred'])

## POST-PROCESSING #2 - sum up to regional revenues

# load firm revenue shares
firm_revenue_shares = pd.read_excel('firm-revenue-share.xlsx', sheet_name='shares')
firm_revenue_regions = pd.read_excel('firm-revenue-share.xlsx', sheet_name='regions')

# Calculate the total revenue for each firm-category-year group
total_revenue = TT_reduced.groupby(['firm','iso_o', 'category', 'year'])['normalized_trade_value_pred'].sum().reset_index(name='total_revenue')
total_not_revenue = TT_reduced.groupby(['firm','iso_o', 'category', 'year'])['trade_value_pred'].sum().reset_index(name='total_not_revenue')


# Step 1. calculate revenue share for apps and games

# Step 1.1: Merge this total revenue back into TT_reduced
TT_reduced = TT_reduced.merge(total_revenue, on=['firm', 'iso_o', 'category', 'year'])
TT_reduced = TT_reduced.merge(total_not_revenue, on=['firm', 'iso_o', 'category', 'year'])
app_game_mask = TT_reduced['category'].isin(['apps', 'games'])

# Step 1.2: Use the mask with .loc to update 'total_not_revenue' where the condition is True
TT_reduced.loc[app_game_mask, 'total_not_revenue'] = TT_reduced.loc[app_game_mask, 'total_revenue'].astype('float32')

# Step 1.3: Calculate the revenue_share
TT_reduced['revenue_share'] = TT_reduced['trade_value_pred'] / TT_reduced['total_not_revenue']
TT_reduced.loc[app_game_mask, 'revenue_share'] = (TT_reduced.loc[app_game_mask, 'normalized_trade_value_pred'] / TT_reduced.loc[app_game_mask, 'total_revenue']).astype('float32')



# Step 2: Adjust for countries with known consumption

# Step 2.1: Merge TT_reduced with firm_revenue_shares
TT_reduced = pd.merge(TT_reduced, firm_revenue_shares, how='left', left_on=['firm', 'iso_d'], right_on=['firm', 'region_iso_d'], suffixes=('', '_frs'))

# Step 2.2: Compute the final_revenue_share
TT_reduced['final_revenue_share'] = TT_reduced['revenue_share_frs'].where(pd.notnull(TT_reduced['revenue_share_frs']), None)

# Drop the extra columns introduced by the merge
TT_reduced.drop(['region_iso_d', 'revenue_share_frs'], axis=1, inplace=True)


# Step 3. Adjust for regions with known consumption

# Step 3.1: Pre-compute the mapping from regions to countries
region_to_countries = {region: firm_revenue_regions[region].dropna().unique().tolist()
                       for region in firm_revenue_regions.columns}

# Step 3.2: Identify rows in firm_revenue_shares not included in the first step
unprocessed_frs = firm_revenue_shares[~firm_revenue_shares['region_iso_d'].isin(TT_reduced['iso_d'].unique())]


# Calculate the total number of iterations for progress tracking
total_iterations = len(unprocessed_frs)

# Step 3.3: Iterate over unprocessed firm_revenue_shares and renormalize revenue shares
for i, (index, frs_row) in enumerate(unprocessed_frs.iterrows(), 1):
    firm = frs_row['firm']
    region_iso_d = frs_row['region_iso_d']
    target_revenue_share = frs_row['revenue_share']

    if region_iso_d in region_to_countries:
        region_vals = region_to_countries[region_iso_d]

        # Filter TT_reduced for the current firm and get unique categories
        firm_categories = TT_reduced[TT_reduced['firm'] == firm]['category'].unique()

        for year in TT_reduced['year'].unique():
            for category in firm_categories:
                mask = (TT_reduced['firm'] == firm) & (TT_reduced['year'] == year) & \
                       (TT_reduced['category'] == category) & (TT_reduced['iso_d'].isin(region_vals))

                relevant_rows = TT_reduced[mask]
                total_share = relevant_rows['revenue_share'].sum()

                if total_share > 0:
                    scaling_factor = target_revenue_share / total_share
                    TT_reduced.loc[mask, 'final_revenue_share'] = relevant_rows['revenue_share'] * scaling_factor

    # Display the percentage completed
    percentage_completed = (i / total_iterations) * 100
    print(f"Completed: {percentage_completed:.2f}%")

# Step 4: Normalize the values for each region/country and product with unknown regional data

# Step 4.1: Identify firms in TT_reduced not present in firm_revenue_shares
unique_firms_in_frs = firm_revenue_shares['firm'].unique()
firms_not_in_frs = TT_reduced.loc[~TT_reduced['firm'].isin(unique_firms_in_frs), 'firm'].unique()

# Step 4.2: 
# Set final_revenue_share equal to revenue_share for these firms
TT_reduced.loc[TT_reduced['firm'].isin(firms_not_in_frs), 'final_revenue_share'] = TT_reduced['revenue_share']

# Group by 'firm', 'category', 'year' and calculate the sum of 'final_revenue_share' 
# Step 4.3: Identify groups with at least one NaN in final_revenue_share
groups_with_nan = TT_reduced[TT_reduced['final_revenue_share'].isna()].groupby(['firm', 'iso_o', 'category', 'year']).size().reset_index(name='count').drop(columns='count')

# Step 4.4: Calculate the sum of final_revenue_share for each group, excluding NaN values
grouped_sum = TT_reduced.groupby(['firm', 'iso_o', 'category', 'year'])['final_revenue_share'].sum(min_count=1).reset_index()

# Step 4.5: Merge to include only groups identified in step 4.4
grouped_sum = groups_with_nan.merge(grouped_sum, on=['firm', 'iso_o', 'category', 'year'], how='left')

# Merge this back to TT_reduced only for rows where 'final_revenue_share' is NaN
TT_reduced = TT_reduced.merge(grouped_sum, on=['firm', 'iso_o', 'category', 'year'], how='left', suffixes=('', '_group_sum'))

# Calculate the total number of iterations for progress tracking
total_iterations = len(grouped_sum)

# Iterate over each group and normalize NaN entries in 'final_revenue_share'
for i, row in enumerate(grouped_sum.iterrows(), 1):
    firm = row[1]['firm']
    category = row[1]['category']
    year = row[1]['year']
    group_sum = row[1]['final_revenue_share']

    mask = (TT_reduced['firm'] == firm) & (TT_reduced['category'] == category) & (TT_reduced['year'] == year) & TT_reduced['final_revenue_share'].isna()
    
    # Calculate the sum of 'revenue_share' for NaN entries
    nan_sum = TT_reduced.loc[mask, 'revenue_share'].sum()

    # Normalize NaN entries
    if nan_sum > 0:
        normalization_factor = (1 - group_sum) / nan_sum
        TT_reduced.loc[mask, 'final_revenue_share'] = TT_reduced.loc[mask, 'revenue_share'] * normalization_factor

    # Display the percentage completed
    percentage_completed = (i / total_iterations) * 100
    print(f"Completed: {percentage_completed:.2f}%")

# Drop the extra '_group_sum' column
TT_reduced.drop('final_revenue_share_group_sum', axis=1, inplace=True)

TT_reduced['final_revenue_share'] = TT_reduced['final_revenue_share'].apply(lambda x: 0 if x < 10**-8 else x)

estimated_rev_share = TT_reduced.groupby(['firm','iso_o', 'category', 'year'])['final_revenue_share'].sum().reset_index(name='estimated_rev_share')
TT_reduced = TT_reduced.merge(estimated_rev_share, on=['firm', 'iso_o', 'category', 'year'])
TT_reduced['final_revenue_share'] = TT_reduced['final_revenue_share'] / TT_reduced['estimated_rev_share']
TT_reduced['normalized_trade_value_pred'] = TT_reduced['final_revenue_share'] * TT_reduced['total_revenue']
TT_reduced['year'] = TT_reduced['year'].astype(int)

In [ ]:
## PLOT REVENUE SHARE

# Assuming TT_reduced_2, firm_revenue_shares, and firm_revenue_regions are already loaded DataFrames
TT_reduced_2 = TT_reduced.copy()
firm_revenue_shares = pd.read_excel('firm-revenue-share.xlsx', sheet_name='shares')

# Filter TT_reduced_2 to include only firms present in firm_revenue_shares
relevant_firms = firm_revenue_shares['firm'].unique()
TT_reduced_2_filtered = TT_reduced_2[TT_reduced_2['firm'].isin(relevant_firms)]

# Filter TT_reduced_2 to include only data for the year 2021
TT_reduced_2021 = TT_reduced_2_filtered[TT_reduced_2_filtered['year'] == 2021]

# Group by 'firm' and sum up 'normalized_trade_value_pred' and 'normalized_trade_value_pred_linear' across all categories
firm_summaries = TT_reduced_2021.groupby(['firm','iso_d'])[['trade_value_pred', 'trade_value_linear_pred','normalized_trade_value_pred']].sum().reset_index()

# Step 1: Calculate Total Revenue per Firm
total_revenue = firm_summaries.groupby('firm')[['trade_value_pred', 'trade_value_linear_pred', 'normalized_trade_value_pred']].sum()

# Step 2: Calculate Revenue Share for Each Region
def calculate_revenue_share(row, summaries, total_revenue, column_name):
    firm = row['firm']
    region_iso_d = row['region_iso_d']
    if region_iso_d in summaries['iso_d'].unique():
        region_sum = summaries[(summaries['firm'] == firm) & (summaries['iso_d'] == region_iso_d)][column_name].sum()
    else:
        # Use the firm_revenue_regions DataFrame
        alternative_regions = firm_revenue_regions[region_iso_d].unique()
        region_sum = summaries[(summaries['firm'] == firm) & (summaries['iso_d'].isin(alternative_regions))][column_name].sum()
    total_firm_revenue = total_revenue.loc[firm, column_name]
    return region_sum / total_firm_revenue if total_firm_revenue != 0 else 0

firm_revenue_shares['revenue_share_predicted'] = firm_revenue_shares.apply(lambda row: calculate_revenue_share(row, firm_summaries, total_revenue, 'trade_value_pred'), axis=1)
firm_revenue_shares['revenue_share_linear'] = firm_revenue_shares.apply(lambda row: calculate_revenue_share(row, firm_summaries, total_revenue, 'trade_value_linear_pred'), axis=1)
firm_revenue_shares['revenue_share_final'] = firm_revenue_shares.apply(lambda row: calculate_revenue_share(row, firm_summaries, total_revenue, 'normalized_trade_value_pred'), axis=1)

# Step 3: The firm_revenue_shares DataFrame now has the new columns 'revenue_share' and 'revenue_share_linear' added

# Creating the scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='revenue_share', y='revenue_share_predicted', data=firm_revenue_shares)

# Calculating the correlation
correlation = firm_revenue_shares['revenue_share'].corr(firm_revenue_shares['revenue_share_predicted'])
# Calculating the R-squared value

# Adding the title with the correlation value
plt.title(f'Scatter Plot with Correlation: {correlation:.2f}')
plt.xlabel('Revenue Share')
plt.ylabel('Revenue Share Predicted')
plt.grid(True)
plt.show()

firm_revenue_shares.to_csv('predicted_shares.csv')

In [ ]:
## OPTIMAL TRANSPORT

# Load distance data
data_distance = pd.read_excel('dist_data.xlsx')
data_distance = data_distance[['iso_o', 'iso_d', 'dist']]

# Set 'dist' to 1 where 'iso_o' is equal to 'iso_d'
data_distance.loc[data_distance['iso_o'] == data_distance['iso_d'], 'dist'] = 0

# Load firm revenues by subsidiary_data
firm_subsidiary_revenues = pd.read_csv('firm_revenues_by_subsidiary.csv')
    
def dpt_optimal_transport(Q, R, D):
    # Normalize revenues and consumption
    R = 1e4 * R / np.sum(R)
    Q = 1e4 * Q / np.sum(Q)

    # Reshape and normalize Phi_vec
    Phi_vec = D.flatten()
    Phi_vec = Phi_vec

    # Create sparse matrices for constraints
    num_R = len(R)
    num_Q = len(Q)
    Aeq_top = kron(csr_matrix(np.ones((1, num_R))), eye(num_Q))
    Aeq_bottom = kron(eye(num_R), csr_matrix(np.ones((1, num_Q))))
    Aeq = vstack([Aeq_top, Aeq_bottom], format='csr')
    beq = np.concatenate([Q, R])

    # Define bounds, and linear programming
    W0 = np.ones_like(Phi_vec) / len(Phi_vec)
    lb = np.zeros_like(W0)
    ub = 1e4 * np.ones_like(W0)
    res = linprog(-Phi_vec, A_eq=Aeq, b_eq=beq, bounds=list(zip(lb, ub)), method='highs')
    
    if not res.success:
        raise ValueError(f"Linear programming did not succeed: {res.message}")


    # Reshape the result
    W_opt = res.x.reshape((num_R, num_Q))

    return W_opt

def estimate_optimal_transport(year, TT, data_distance):
    TT_reduced = TT[TT['year'] == year].copy()

    TT_results_v2 = firm_subsidiary_revenues[firm_subsidiary_revenues['year'] == year].copy()

    # Selecting the columns by their Python indices
    selected_columns = TT_results_v2.iloc[:, [2, 3, 5]]

    # Getting unique rows (combinations) from these columns
    parents_categories = selected_columns.drop_duplicates()
    # Preprocess data_distance for quick lookup

    # Create a dictionary to map (iso_o, iso_d) to dist
    dist_dict = {(row['iso_o'], row['iso_d']): row['dist'] for _, row in data_distance.iterrows()}

    all_TT_optimal = []  # List to store results for each i
    total_length = len(parents_categories)  # Total number of iterations
    optimized_firms_categories = []  # List to store firms and categories for which optimization was done

    
    for i in range(len(parents_categories)):
        print(f"Processing {i+1}/{total_length} ({(i+1)/total_length*100:.2f}%)")

        parent = parents_categories.iloc[i]['parent']
        category = parents_categories.iloc[i]['category']
    
        z_TT = TT_reduced[(TT_reduced['firm'] == parent) & (TT_reduced['category'] == category)]
        z_TT_results = TT_results_v2[(TT_results_v2['parent'] == parent) & (TT_results_v2['category'] == category)]

        # Group by 'iso_o' and sum 'value' for each iso_o, then remove entries with sum 0
        rev_TT_results = z_TT_results.groupby('iso_o')['value'].sum()
        rev_TT_results = rev_TT_results[rev_TT_results != 0]
        iso_o = rev_TT_results.index

        # Group by 'iso_d' and sum 'normalized_trade_value_pred' for each iso_d, then remove entries with sum 0
        rev_TT = z_TT.groupby('iso_d')['normalized_trade_value_pred'].sum()
        rev_TT = rev_TT[rev_TT != 0]
        iso_d = rev_TT.index
        

        if len(iso_o) > 1 and rev_TT.sum() >0:
            
            # Perform operations if there are more than one unique iso_o values
            optimized_firms_categories.append((parent, category))


            # Initialize and fill the cost matrix
            cost_matrix = np.full((len(iso_o), len(iso_d)), np.nan)
            for idx_o, o in enumerate(iso_o):
                for idx_d, d in enumerate(iso_d):
                    dist = dist_dict.get((o, d))
                    if dist is not None:
                        cost_matrix[idx_o, idx_d] = 1 / (1 + dist)
                               
            cost_matrix *= 10**4

            # Optimal transport calculation
            W_max = max(sum(rev_TT), sum(rev_TT_results))
            W_opt = W_max * dpt_optimal_transport(rev_TT, rev_TT_results, cost_matrix) / 10**4 
            

            # Data preparation for DataFrame
            firm_vals = np.repeat(parent, len(iso_o) * len(iso_d))
            year_vals = np.repeat(parents_categories.iloc[i]['year'], len(iso_o) * len(iso_d))
            category_vals = np.repeat(category, len(iso_o) * len(iso_d))
            normalized_trade_value_pred = W_opt.flatten()
            iso_o_v2 = np.tile(iso_o, (len(iso_d), 1)).T.flatten()
            iso_d_v2 = np.tile(iso_d, (len(iso_o), 1)).flatten()

            # Creating and storing the DataFrame
            TT_optimal = pd.DataFrame({
                'firm': firm_vals,
                'year': year_vals,
                'category': category_vals,
                'iso_o': iso_o_v2,
                'iso_d': iso_d_v2,
                'normalized_trade_value_pred': normalized_trade_value_pred
            })
            all_TT_optimal.append(TT_optimal)

    # Concatenate all DataFrames in the list to a single DataFrame
    all_TT_optimal_df = pd.concat(all_TT_optimal, ignore_index=True)

    columns_to_keep = ['firm', 'year', 'category', 'iso_o', 'iso_d', 'normalized_trade_value_pred']

    # Select only these columns
    TT_reduced = TT_reduced[columns_to_keep]


    # Convert the list to a DataFrame for easier processing
    optimized_df = pd.DataFrame(optimized_firms_categories, columns=['parent', 'category'])

    # Create a condition to identify rows to be removed from TT_reduced
    condition_df = TT_reduced[['firm', 'category']].merge(optimized_df, left_on=['firm', 'category'], right_on=['parent', 'category'], how='left', indicator=True)
    condition_to_remove = condition_df['_merge'] == 'both'

    # Reset the index of TT_reduced to align with the condition
    TT_reduced.reset_index(drop=True, inplace=True)

    # Filter out these rows from TT_reduced
    TT_reduced_filtered = TT_reduced[~condition_to_remove.values]

    # Concatenate TT_reduced_filtered with all_TT_optimal_df
    final_result_df = pd.concat([TT_reduced_filtered, all_TT_optimal_df], ignore_index=True)

    # At the end of the code, return final_result_df for the year
    return final_result_df

final_results_ot = {}
for year in range(2016, 2022):
    final_results_ot[year] = estimate_optimal_transport(year,TT_reduced,data_distance)
    
final_results_hq = {}
for year in range(2016, 2022):
    # Filter TT_reduced for the current year and select specific columns
    final_results_hq[year] = TT_reduced[
        (TT_reduced['year'] == year)
    ][['firm', 'year', 'category', 'iso_o', 'iso_d', 'normalized_trade_value_pred']]

In [ ]:
## CALCULATE CONFIDENCE INTERVALS & EXPORT FINAL RESULTS

def calculate_confidence_interval(TT):
    TT = TT[TT['normalized_trade_value_pred'] > 0]

    # Grouping by 'firm' and 'iso_o' and calculating the sum
    grouped_sum = TT.groupby(['firm', 'iso_o', 'category'])['normalized_trade_value_pred'].sum().reset_index()
    grouped_sum = grouped_sum.rename(columns={'normalized_trade_value_pred': 'total_revenues'})

    # Merging the sum back into the original DataFrame
    TT = TT.merge(grouped_sum, on=['firm', 'iso_o', 'category'], how='left')

    # Transforming the target variable
    TT['log_normalized_trade_value_pred'] = np.log(1 + TT['normalized_trade_value_pred'])
    TT['log_total_revenues'] = np.log(1 + TT['total_revenues'])

    # Convert 'iso_o' to a categorical type with the baseline level first
    TT['iso_o'] = pd.Categorical(TT['iso_o'], categories=['USA'] + [x for x in TT['iso_o'].unique() if x != 'USA'], ordered=True)


    # Adjusted approach to fit the initial model using the reordered 'iso_o'
    initial_formula = 'log_normalized_trade_value_pred ~ log_total_revenues + C(iso_o, Treatment(reference="USA")) + C(iso_d, Treatment(reference="USA"))'
    initial_model = smf.ols(initial_formula, data=TT).fit()

    # Identify significant levels in iso_o (p-value < 0.05)
    significant_iso_o_levels = ['USA']  # Dynamically selected baseline included

    for level in TT['iso_o'].cat.categories:  # Corrected access to categories
        if level == 'USA':
            continue
        else:
            coefficient_name = f'C(iso_o, Treatment(reference="USA"))[T.{level}]'
            if coefficient_name in initial_model.pvalues and initial_model.pvalues[coefficient_name] < 0.05:
                significant_iso_o_levels.append(level)

    # Recoding insignificant levels to 'Other'
    TT['iso_o_reduced'] = TT['iso_o'].apply(lambda x: x if x in significant_iso_o_levels else 'Other').astype('category')

    # Fit the model using the modified iso_o column
    formula = 'log_normalized_trade_value_pred ~ log_total_revenues + C(iso_o_reduced, Treatment(reference="USA")) + C(iso_d, Treatment(reference="USA"))'
    model = smf.ols(formula, data=TT).fit()

    # Predicting values
    TT['predicted'] = model.predict()

    # Calculating confidence intervals
    predictions = model.get_prediction(TT)
    ci = predictions.summary_frame(alpha=0.05)[['mean_ci_lower', 'mean_ci_upper']]

    # Adding confidence intervals back to the DataFrame
    ci.reset_index(drop=True, inplace=True)
    TT.reset_index(drop=True, inplace=True)
    TT = pd.concat([TT, ci], axis=1)

    # Adjustments for predicted values and confidence intervals
    TT['predicted'] = np.exp(TT['predicted']) - 1
    TT['mean_ci_lower'] = (np.exp(TT['mean_ci_lower']) - 1) * TT['normalized_trade_value_pred'] / TT['predicted']
    TT['mean_ci_upper'] = (np.exp(TT['mean_ci_upper']) - 1) * TT['normalized_trade_value_pred'] / TT['predicted']

    # Selecting and renaming specific columns
    TT_merged = TT[['firm', 'iso_o', 'iso_d', 'category', 'year', 'normalized_trade_value_pred', 'mean_ci_lower', 'mean_ci_upper']].copy()
    TT_merged.rename(columns={
        'normalized_trade_value_pred': 'trade_value_pred',
        'mean_ci_lower': 'lb_value',
        'mean_ci_upper': 'ub_value'
    }, inplace=True)

    return TT_merged


final_results_ot_intervals = {}
columns_to_sum = ['trade_value_pred', 'ub_value', 'lb_value']  # Ensure these columns exist in your DataFrame
for year in range(2016, 2022):
    final_results_ot_intervals[year] = calculate_confidence_interval(final_results_ot[year])
    df = final_results_ot_intervals[year].copy()
    df = df[df['iso_o'] != df['iso_d']]
    df.to_csv(f"firm_trade_ot_intervals_{year}.csv", index=False)
    df_aggregated = df.groupby(['iso_o', 'iso_d','year','category'], observed=False)[columns_to_sum].sum().reset_index()
    df_aggregated.to_csv(f"country_trade_ot_intervals_{year}.csv", index=False)
    print(year)

final_results_hq_intervals = {}
for year in range(2016, 2022):
    final_results_hq_intervals[year] = calculate_confidence_interval(final_results_hq[year])   
    df = final_results_hq_intervals[year].copy()
    df = df[df['iso_o'] != df['iso_d']]
    df.to_csv(f"firm_trade_hq_intervals_{year}.csv", index=False)
    df_aggregated = df.groupby(['iso_o', 'iso_d','year','category'], observed=False)[columns_to_sum].sum().reset_index()
    df_aggregated.to_csv(f"country_trade_hq_intervals_{year}.csv", index=False)
    print(year)

In [ ]:
## EXPORT THE DATA AS FOUR LARGE CSV FILES

# Your code modification starts here
columns_to_sum = ['trade_value_pred', 'ub_value', 'lb_value']

# Initialize empty DataFrames for concatenation
all_firm_trade_ot = pd.DataFrame()
all_country_trade_ot = pd.DataFrame()
all_firm_trade_hq = pd.DataFrame()
all_country_trade_hq = pd.DataFrame()

for year in range(2016, 2022):
    # OT Data
    df_ot = calculate_confidence_interval(final_results_ot[year])
    df_ot = df_ot[df_ot['iso_o'] != df_ot['iso_d']]
    df_ot = df_ot[df_ot['trade_value_pred'] != 0]
    all_firm_trade_ot = pd.concat([all_firm_trade_ot, df_ot], ignore_index=True)
    
    # Aggregate OT data
    df_aggregated_ot = df_ot.groupby(['iso_o', 'iso_d', 'year', 'category'], observed=False)[columns_to_sum].sum().reset_index()
    df_aggregated_ot = df_aggregated_ot[df_aggregated_ot['trade_value_pred'] != 0]
    all_country_trade_ot = pd.concat([all_country_trade_ot, df_aggregated_ot], ignore_index=True)
    
    # HQ Data
    df_hq = calculate_confidence_interval(final_results_hq[year])
    df_hq = df_hq[df_hq['iso_o'] != df_hq['iso_d']]
    df_hq = df_hq[df_hq['trade_value_pred'] != 0]
    all_firm_trade_hq = pd.concat([all_firm_trade_hq, df_hq], ignore_index=True)
    
    # Aggregate HQ data
    df_aggregated_hq = df_hq.groupby(['iso_o', 'iso_d', 'year', 'category'], observed=False)[columns_to_sum].sum().reset_index()
    df_aggregated_hq = df_aggregated_hq[df_aggregated_hq['trade_value_pred'] != 0]
    all_country_trade_hq = pd.concat([all_country_trade_hq, df_aggregated_hq], ignore_index=True)

# Export to CSV
all_firm_trade_ot.to_csv("all_firm_trade_ot_intervals.csv", index=False)
all_country_trade_ot.to_csv("all_country_trade_ot_intervals.csv", index=False)
all_firm_trade_hq.to_csv("all_firm_trade_hq_intervals.csv", index=False)
all_country_trade_hq.to_csv("all_country_trade_hq_intervals.csv", index=False)

print("CSV files have been exported.")

In [ ]:
## BUBBLE CHART DATA

# List of countries in EU-27
eu_countries = ['AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE']

# Function to process DataFrame
def process_data(df):
    # Remove rows where iso_o is equal to iso_d
    df = df[df['iso_o'] != df['iso_d']].copy()

    # Replace iso_o with 'EU-27' for EU countries
    df['iso_o'] = df['iso_o'].apply(lambda x: 'EU-27' if x in eu_countries else x)
    
    pivot_df = df.groupby(['firm', 'iso_o'])['trade_value_pred'].sum().reset_index()
    
    filtered_aggregated_df = pivot_df[pivot_df['trade_value_pred'] > 0]

    # Aggregate trade_value_pred by firm and iso_o
    return filtered_aggregated_df

# Processing final_results_ot_intervals and final_results_hq_intervals for the year 2021
aggregated_ot_2021 = process_data(final_results_ot_intervals[2021])
aggregated_hq_2021 = process_data(final_results_hq_intervals[2021])

# Exporting to Excel
with pd.ExcelWriter('aggregated_trade_values_2021.xlsx') as writer:
    aggregated_ot_2021.to_excel(writer, sheet_name='OT_2021', index=False)
    aggregated_hq_2021.to_excel(writer, sheet_name='HQ_2021', index=False)

In [ ]:
## EXPORT THE LIST OF FIRMS AND APPS/GAMES

# Filter the DataFrame for rows where category is 'apps' or 'games'
filtered_df = TT_reduced[TT_reduced['category'].isin(['apps', 'games'])]

# Finding unique entries in the columns 'firm', 'category', 'iso_o'
unique_entries = filtered_df[['firm', 'category', 'iso_o']].drop_duplicates()

# Exporting to CSV
unique_entries.to_csv('apps_unique_category_entries.csv', index=False)

# Finding unique entries in the columns 'firm', 'category', 'iso_o'
unique_entries = filtered_df[['firm', 'iso_o']].drop_duplicates()

# Exporting to CSV
unique_entries.to_csv('apps_unique_entries.csv', index=False)


# Finding unique rows for specified columns
unique_rows = firm_subsidiary_revenues[['firm', 'iso_o', 'category', 'parent']].drop_duplicates()

# Exporting to CSV
output_file = 'firms_unique_category_entries.csv'
unique_rows.to_csv(output_file, index=False)


# Finding unique rows for specified columns
unique_rows = firm_subsidiary_revenues[['firm', 'iso_o', 'parent']].drop_duplicates()

# Exporting to CSV
output_file = 'firms_unique_entries.csv'
unique_rows.to_csv(output_file, index=False)